# 🎓 ATLAS: Academic Task and Learning Agent System

## 학습 노트북 - LangGraph 멀티 에이전트 시스템

---

### 📚 이 노트북에서 배울 내용

1. **ReACT (Reasoning + Acting) 패턴**: AI 에이전트가 사고하고 행동하는 구조화된 방법
2. **멀티 에이전트 시스템**: 여러 전문 에이전트가 협업하는 시스템 설계
3. **LangGraph**: 복잡한 AI 워크플로우를 그래프로 표현하는 프레임워크
4. **비동기 프로그래밍**: asyncio를 활용한 효율적인 병렬 처리

---

### 🎯 ATLAS 시스템 개요

ATLAS는 학생들의 학업 관리를 돕는 **지능형 멀티 에이전트 시스템**입니다.

#### 핵심 에이전트 구성

```
┌─────────────────────┐
│  Coordinator Agent  │  ← 마스터 에이전트 (조율)
└──────────┬──────────┘
           │
     ┌─────┴─────┬─────────────┐
     │           │             │
┌────▼────┐ ┌───▼─────┐ ┌────▼──────┐
│ Planner │ │NoteWriter│ │  Advisor  │
│  Agent  │ │  Agent   │ │   Agent   │
└─────────┘ └──────────┘ └───────────┘
```

- **Coordinator**: 다른 에이전트들을 조율하고 작업 분배
- **Planner**: 일정 관리 및 학습 계획 수립
- **NoteWriter**: 학습 자료 생성 및 요약
- **Advisor**: 개인화된 학습 조언 제공

---

## 1️⃣ 필수 라이브러리 설치

### 왜 이 라이브러리들이 필요한가?

#### 🔧 핵심 라이브러리와 역할

| 라이브러리 | 역할 | 중요한 이유 |
|-----------|------|------------|
| **langgraph** | AI 워크플로우를 그래프로 표현 | 에이전트 간 데이터 흐름과 제어 흐름을 명확하게 관리 |
| **langchain** | LLM 통합 및 프롬프트 관리 | 구조화된 메시지 처리와 체인 구성 |
| **openai** | OpenAI 호환 API 호출 | NVIDIA NeMo를 OpenAI 인터페이스로 사용 |
| **pydantic** | 데이터 검증 및 타입 안정성 | 런타임 에러 방지 및 타입 체크 |
| **typing-extensions** | 고급 타입 힌팅 | TypedDict로 구조화된 딕셔너리 정의 |

---

In [ ]:
# 필수 패키지 설치
# Colab 환경이 아니라면 터미널에서 실행하세요

!pip install -q langgraph langchain langchain-core openai pydantic typing-extensions

## 2️⃣ 라이브러리 Import

### 각 라이브러리의 용도

```python
# 비동기 프로그래밍
import asyncio  # 여러 에이전트를 동시에 실행하기 위한 비동기 처리

# LangGraph
from langgraph.graph import StateGraph, END  # 상태 기반 워크플로우 그래프 구성

# LangChain  
from langchain_core.messages import HumanMessage, AIMessage  # 대화 구조화

# OpenAI (NVIDIA NeMo 호출용)
from openai import AsyncOpenAI  # 비동기 LLM API 호출

# Pydantic
from pydantic import BaseModel  # 데이터 모델 검증
```

---

In [ ]:
# ========================================
# 필수 라이브러리 Import
# ========================================

# === 비동기 프로그래밍 ===
# asyncio: 여러 에이전트를 동시에 실행하여 성능 최적화
import asyncio
from typing import List, Dict, Optional, Any

# === 날짜/시간 처리 ===
# 일정 관리에 필수적인 datetime 모듈
from datetime import datetime, timezone, timedelta

# === 데이터 처리 ===
# JSON 형식의 데이터 파싱 및 생성
import json

# === LangGraph: 워크플로우 구성 ===
# StateGraph: 상태를 공유하며 흐르는 그래프 구조
# END: 그래프의 종료 노드
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# === LangChain: LLM 메시지 처리 ===
# 사용자와 AI의 대화를 구조화
from langchain_core.messages import HumanMessage, AIMessage

# === OpenAI API ===
# NVIDIA NeMo 모델을 OpenAI 호환 인터페이스로 호출
from openai import AsyncOpenAI

# === Pydantic: 데이터 검증 ===
# BaseModel을 상속하여 타입 안정성 확보
from pydantic import BaseModel

print("✅ 모든 라이브러리 임포트 완료")

In [ ]:
## 3️⃣ API 키 설정

### 🔑 NVIDIA API 키 발급 방법

1. [NVIDIA NGC](https://build.nvidia.com) 접속
2. 무료 계정 생성
3. API 키 발급
4. 아래 셀에 입력

### ⚠️ 보안 주의사항

- API 키는 절대 공개 저장소에 올리지 마세요
- 환경 변수나 별도 설정 파일 사용 권장
- Colab에서는 Secrets 기능 활용

---

In [1]:
# ========================================
# API 키 설정
# ========================================

# 방법 1: 직접 입력 (학습용)
NVIDIA_API_KEY = "your-api-key-here"

# 방법 2: 환경 변수 사용 (권장)
# import os
# NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")

# 방법 3: Google Colab Secrets
# from google.colab import userdata
# NVIDIA_API_KEY = userdata.get('NVIDIA_API_KEY')

if NVIDIA_API_KEY == "your-api-key-here":
    print("⚠️  위 셀에 실제 API 키를 입력해주세요!")
else:
    print("✅ API 키 설정 완료")

⚠️  위 셀에 실제 API 키를 입력해주세요!


## 4️⃣ 데이터 모델 정의

### 📋 TypedDict vs 일반 Dict

#### 일반 Dict의 문제점
```python
state = {"messages": [], "profile": {}}
# 오타가 있어도 실행 시까지 모름
print(state["mesages"])  # KeyError!
```

#### TypedDict의 장점
```python
class State(TypedDict):
    messages: List[Any]
    profile: Dict

# IDE가 자동완성 제공
# 타입 체커가 오타 미리 발견
```

### 🎯 AcademicState 구조

- `messages`: 사용자-AI 대화 히스토리
- `profile`: 학생 프로필 (학습 스타일, 전공 등)
- `calendar`: 캘린더 이벤트 (시험, 과제 마감일)
- `tasks`: 할 일 목록
- `results`: 에이전트 실행 결과

---

In [ ]:
# ========================================
# 핵심 데이터 모델 정의
# ========================================

class AcademicState(TypedDict):
    """
    학업 관리 시스템의 전체 상태를 나타내는 타입
    
    이 상태는 LangGraph의 모든 노드(에이전트)를 거치며 전달됩니다.
    각 에이전트는 이 상태를 읽고 업데이트할 수 있습니다.
    
    TypedDict를 사용하는 이유:
    1. 타입 안정성: 잘못된 키 사용 방지
    2. IDE 지원: 자동완성 및 타입 힌트
    3. 문서화: 상태 구조를 명확히 표현
    """
    messages: List[Any]      # 대화 히스토리 (HumanMessage, AIMessage)
    profile: Dict            # 학생 프로필 (학습 선호도, 전공 등)
    calendar: Dict           # 캘린더 이벤트 (시험, 과제 마감일)
    tasks: Dict              # 할 일 목록
    results: Dict            # 에이전트 실행 결과 저장소


# ========================================
# Pydantic 모델: 강력한 데이터 검증
# ========================================

class AgentAction(BaseModel):
    """
    에이전트의 '행동'을 정의하는 모델
    
    ReACT 패턴의 'Action' 단계를 구조화합니다.
    Pydantic이 자동으로 타입 검증 및 변환을 수행합니다.
    
    사용 예시:
        action = AgentAction(
            action="search_calendar",
            thought="시험 일정을 확인해야 함",
            tool="calendar_search"
        )
    """
    action: str                          # 행동 이름 (예: "search_calendar")
    thought: str                         # 행동의 이유 (ReACT의 사고 과정)
    tool: Optional[str] = None           # 사용할 도구 (선택사항)
    action_input: Optional[Dict] = None  # 행동에 필요한 파라미터


class AgentOutput(BaseModel):
    """
    에이전트의 '관찰 결과'를 정의하는 모델
    
    ReACT 패턴의 'Observation' 단계를 구조화합니다.
    
    사용 예시:
        output = AgentOutput(
            observation="3개의 예정된 이벤트를 찾았습니다",
            output={"events": [...], "count": 3}
        )
    """
    observation: str    # 관찰한 내용 (자연어)
    output: Dict        # 구조화된 출력 데이터

print("✅ 데이터 모델 정의 완료")

## 5️⃣ LLM 설정: NVIDIA NeMo

### 🤖 왜 NVIDIA NeMo인가?

1. **고성능**: 340B 파라미터의 대규모 모델
2. **무료 API**: 일정량까지 무료 사용 가능
3. **OpenAI 호환**: 익숙한 인터페이스
4. **비동기 지원**: 병렬 처리에 최적화

### 📊 주요 파라미터 설명

- **temperature**: 응답의 창의성 조절
  - 0.0: 매우 결정적 (같은 입력 → 같은 출력)
  - 0.5: 균형잡힌 (기본값)
  - 1.0: 매우 창의적 (다양한 응답)

- **max_tokens**: 응답 최대 길이
  - 짧은 답변: 256
  - 중간 길이: 1024 (기본값)
  - 긴 답변: 2048+

---

In [ ]:
# ========================================
# LLM 설정 클래스
# ========================================

class LLMConfig:
    """
    LLM(대형 언어 모델)의 설정을 중앙에서 관리
    
    클래스 변수를 사용하여 모든 인스턴스가 동일한 설정을 공유합니다.
    필요시 인스턴스별로 다른 설정도 가능합니다.
    """
    base_url: str = "https://integrate.api.nvidia.com/v1"  # NVIDIA API 엔드포인트
    model: str = "nvidia/nemotron-4-340b-instruct"         # 모델 이름
    max_tokens: int = 1024                                 # 최대 응답 길이
    default_temp: float = 0.5                              # 기본 temperature


# ========================================
# NeMo LLM 래퍼 클래스
# ========================================

class NeMoLLaMa:
    """
    NVIDIA NeMo 모델과 통신하기 위한 래퍼 클래스
    
    OpenAI의 AsyncOpenAI 클라이언트를 사용하여 NVIDIA API를 호출합니다.
    (NVIDIA API가 OpenAI 호환 인터페이스를 제공하기 때문)
    
    주요 기능:
    - 비동기 텍스트 생성 (agenerate)
    - API 인증 확인 (check_auth)
    - 자동 에러 핸들링
    
    비동기를 사용하는 이유:
    여러 에이전트가 동시에 LLM을 호출할 때 블로킹 없이
    효율적으로 처리할 수 있습니다.
    """
    
    def __init__(self, api_key: str):
        """LLM 클라이언트 초기화"""
        self.config = LLMConfig()
        self.client = AsyncOpenAI(
            base_url=self.config.base_url,
            api_key=api_key
        )
        self._is_authenticated = False
    
    async def check_auth(self) -> bool:
        """
        API 키가 유효한지 확인
        
        간단한 테스트 메시지를 보내서 인증 상태를 확인합니다.
        실제 프로덕션에서는 시작 시 한 번만 호출하면 됩니다.
        """
        test_message = [{"role": "user", "content": "test"}]
        try:
            await self.agenerate(test_message, temperature=0.1)
            self._is_authenticated = True
            print("✅ API 인증 성공")
            return True
        except Exception as e:
            print(f"❌ API 인증 실패: {str(e)}")
            return False
    
    async def agenerate(
        self,
        messages: List[Dict],
        temperature: Optional[float] = None
    ) -> str:
        """
        비동기 텍스트 생성 - 가장 중요한 메서드
        
        모든 에이전트가 이 메서드를 통해 LLM을 호출합니다.
        'async/await'를 사용하여 비동기적으로 실행됩니다.
        
        Args:
            messages: 대화 메시지 리스트
                형식: [{"role": "user", "content": "질문"}, ...]
            temperature: 응답의 창의성 (0.0~1.0)
        
        Returns:
            LLM의 응답 텍스트
        
        사용 예시:
            messages = [
                {"role": "system", "content": "당신은 학습 도우미입니다"},
                {"role": "user", "content": "시험 준비 방법은?"}
            ]
            response = await llm.agenerate(messages, temperature=0.7)
        """
        completion = await self.client.chat.completions.create(
            model=self.config.model,
            messages=messages,
            temperature=temperature or self.config.default_temp,
            max_tokens=self.config.max_tokens,
            stream=False  # 전체 응답을 한 번에 받음
        )
        return completion.choices[0].message.content

print("✅ LLM 클래스 정의 완료")

## 6️⃣ LLM 초기화 및 테스트

### 체크포인트

이 단계에서 확인할 사항:
- ✅ LLM 인스턴스 생성 성공
- ✅ API 키 유효성 확인
- ✅ 간단한 테스트 응답 확인

---

In [ ]:
# ========================================
# LLM 인스턴스 생성
# ========================================

llm = NeMoLLaMa(NVIDIA_API_KEY)

print("✅ LLM 인스턴스 생성 완료")
print(f"📊 모델: {llm.config.model}")
print(f"📏 최대 토큰: {llm.config.max_tokens}")
print(f"🌡️  기본 Temperature: {llm.config.default_temp}")

In [ ]:
# ========================================
# API 인증 테스트
# ========================================

auth_result = await llm.check_auth()

if auth_result:
    print("\n🎉 모든 시스템 정상 작동!")
    print("이제 에이전트를 구축할 준비가 되었습니다.")
else:
    print("\n⚠️  API 키를 다시 확인해주세요.")

## 7️⃣ 샘플 데이터 준비

### 실습용 데이터

실제 환경에서는:
- Google Calendar API에서 일정 가져오기
- Notion API에서 할 일 가져오기
- 학교 시스템에서 프로필 가져오기

학습을 위해 샘플 데이터를 직접 정의합니다.

---

In [ ]:
# ========================================
# 샘플 학생 프로필
# ========================================

sample_profile = {
    "id": "student_001",
    "personal_info": {
        "name": "김철수",
        "major": "컴퓨터공학",
        "academic_year": 3  # 3학년
    },
    "learning_preferences": {
        "learning_style": {
            "primary": "visual",      # 시각적 학습 선호
            "secondary": "hands-on"   # 실습 선호
        },
        "study_patterns": {
            "best_time": "morning",   # 아침에 집중력이 가장 좋음
            "session_length": 90,     # 90분 학습 선호
            "break_length": 15        # 15분 휴식
        }
    },
    "academic_info": {
        "current_courses": [
            {
                "name": "데이터구조",
                "difficulty": "medium",
                "credits": 3
            },
            {
                "name": "알고리즘",
                "difficulty": "hard",
                "credits": 3
            },
            {
                "name": "운영체제",
                "difficulty": "hard",
                "credits": 3
            }
        ]
    }
}

# ========================================
# 샘플 캘린더 데이터
# ========================================

sample_calendar = {
    "events": [
        {
            "summary": "알고리즘 중간고사",
            "start": {"dateTime": "2025-11-01T09:00:00Z"},
            "end": {"dateTime": "2025-11-01T11:00:00Z"},
            "description": "3장~5장 범위"
        },
        {
            "summary": "데이터구조 과제 제출",
            "start": {"dateTime": "2025-10-28T23:59:00Z"},
            "end": {"dateTime": "2025-10-28T23:59:00Z"},
            "description": "이진 탐색 트리 구현"
        },
        {
            "summary": "운영체제 팀 프로젝트 발표",
            "start": {"dateTime": "2025-11-05T14:00:00Z"},
            "end": {"dateTime": "2025-11-05T16:00:00Z"},
            "description": "스케줄링 알고리즘 분석"
        }
    ]
}

# ========================================
# 샘플 할 일 목록
# ========================================

sample_tasks = {
    "tasks": [
        {
            "title": "알고리즘 3장 복습",
            "status": "needsAction",
            "due": "2025-10-25T18:00:00Z",
            "priority": "high"
        },
        {
            "title": "데이터구조 과제 코딩",
            "status": "needsAction",
            "due": "2025-10-27T18:00:00Z",
            "priority": "high"
        },
        {
            "title": "운영체제 PPT 준비",
            "status": "needsAction",
            "due": "2025-11-03T18:00:00Z",
            "priority": "medium"
        }
    ]
}

print("✅ 샘플 데이터 준비 완료\n")
print(f"👤 학생: {sample_profile['personal_info']['name']}")
print(f"📚 전공: {sample_profile['personal_info']['major']}")
print(f"📖 수강 과목: {len(sample_profile['academic_info']['current_courses'])}개")
print(f"📅 예정된 이벤트: {len(sample_calendar['events'])}개")
print(f"✅ 할 일: {len(sample_tasks['tasks'])}개")

## 8️⃣ 실습 1: 간단한 LLM 호출

### 학습 목표

1. 비동기 함수 작성법 (`async def`)
2. 비동기 함수 호출법 (`await`)
3. 프롬프트 구조 이해 (system, user 역할)
4. temperature 파라미터의 효과

---

In [ ]:
# ========================================
# 실습 1: 기본 LLM 호출
# ========================================

async def simple_study_advice():
    """
    간단한 학습 조언을 받는 함수
    
    포인트:
    1. system 메시지: AI의 역할과 성격 정의
    2. user 메시지: 실제 질문이나 요청
    3. temperature: 응답의 다양성 조절
    """
    messages = [
        {
            "role": "system",
            "content": "당신은 대학생들의 학습을 돕는 친절한 AI 멘토입니다. 실용적이고 구체적인 조언을 제공합니다."
        },
        {
            "role": "user",
            "content": "알고리즘 시험이 일주일 남았는데, 효과적으로 준비하는 방법을 알려주세요."
        }
    ]
    
    # LLM 호출 (비동기)
    response = await llm.agenerate(messages, temperature=0.7)
    return response


# 실행
print("🤔 질문: 알고리즘 시험이 일주일 남았는데, 효과적으로 준비하는 방법은?\n")
advice = await simple_study_advice()
print("🤖 AI 멘토의 조언:")
print(advice)

## 9️⃣ 실습 2: Temperature 비교

### Temperature의 효과 체험

같은 질문에 대해 다른 temperature로 호출하여 차이를 확인합니다.

---

In [ ]:
# ========================================
# 실습 2: Temperature 효과 비교
# ========================================

async def compare_temperatures():
    """
    동일한 질문에 대해 다른 temperature로 응답을 비교
    """
    question = "데이터구조를 공부하는 가장 좋은 방법은?"
    
    messages = [
        {"role": "system", "content": "당신은 학습 전문가입니다."},
        {"role": "user", "content": question}
    ]
    
    print(f"📝 질문: {question}\n")
    print("=" * 60)
    
    # Temperature 0.1 (매우 일관적)
    print("\n🌡️  Temperature = 0.1 (일관적/결정적)")
    print("-" * 60)
    response_low = await llm.agenerate(messages, temperature=0.1)
    print(response_low[:200] + "...\n")  # 처음 200자만 출력
    
    # Temperature 0.9 (매우 창의적)
    print("\n🌡️  Temperature = 0.9 (창의적/다양함)")
    print("-" * 60)
    response_high = await llm.agenerate(messages, temperature=0.9)
    print(response_high[:200] + "...\n")  # 처음 200자만 출력

# 실행
await compare_temperatures()

## 🔟 실습 3: 학생 프로필 기반 맞춤 조언

### 컨텍스트를 활용한 개인화

학생의 프로필 정보를 프롬프트에 포함시켜
더 개인화된 조언을 받습니다.

---

In [ ]:
# ========================================
# 실습 3: 프로필 기반 개인화 조언
# ========================================

async def personalized_advice():
    """
    학생 프로필을 고려한 맞춤형 학습 조언
    
    중요: 프롬프트에 학생의 학습 스타일, 선호 시간대 등을
    포함시켜 더 실용적인 조언을 얻습니다.
    """
    # 학생 정보 추출
    student = sample_profile['personal_info']
    preferences = sample_profile['learning_preferences']
    
    # 컨텍스트가 풍부한 프롬프트 작성
    context = f"""
학생 정보:
- 이름: {student['name']}
- 전공: {student['major']}
- 학습 스타일: {preferences['learning_style']['primary']} (시각적 학습 선호)
- 최적 학습 시간: {preferences['study_patterns']['best_time']} (아침)
- 선호 학습 시간: {preferences['study_patterns']['session_length']}분

현재 상황:
- 알고리즘 중간고사 1주일 후
- 데이터구조 과제 3일 후 마감
"""
    
    messages = [
        {
            "role": "system",
            "content": "당신은 학생의 개인적 특성을 고려하여 최적화된 학습 계획을 제안하는 AI 멘토입니다."
        },
        {
            "role": "user",
            "content": f"{context}\n\n이 학생을 위한 이번 주 학습 계획을 구체적으로 제안해주세요."
        }
    ]
    
    response = await llm.agenerate(messages, temperature=0.6)
    return response


# 실행
print("🎯 개인화된 학습 계획\n")
plan = await personalized_advice()
print(plan)

## 🎓 학습 정리

### ✅ 지금까지 배운 내용

1. **라이브러리 설정**
   - LangGraph, LangChain, OpenAI의 역할 이해
   - 각 라이브러리가 필요한 이유

2. **데이터 모델링**
   - TypedDict로 타입 안정성 확보
   - Pydantic으로 데이터 검증

3. **LLM 사용법**
   - 비동기 프로그래밍 (async/await)
   - 프롬프트 구조 (system, user)
   - Temperature 파라미터 조절

4. **개인화 전략**
   - 컨텍스트를 활용한 맞춤형 응답
   - 학생 프로필 기반 조언

---

### 🚀 다음 단계

이 기초를 바탕으로:
1. **ReACT Agent 패턴** 구현
2. **멀티 에이전트 시스템** 구축
3. **LangGraph로 워크플로우** 연결
4. **실제 API 통합** (Google Calendar, Notion 등)

---

### 📚 참고 자료

- [LangGraph 공식 문서](https://langchain-ai.github.io/langgraph/)
- [LangChain 문서](https://python.langchain.com/)
- [NVIDIA NeMo](https://build.nvidia.com)
- [ReACT 논문](https://arxiv.org/abs/2210.03629)
- [Pydantic 문서](https://docs.pydantic.dev/)

---

### 💡 추가 실습 아이디어

1. 여러 질문을 동시에 처리하는 비동기 함수 작성
2. 대화 히스토리를 유지하는 채팅봇 만들기
3. 학생별 프로필을 JSON 파일로 저장/불러오기
4. 다양한 temperature로 응답 품질 비교

---

## 🎉 축하합니다!

LangGraph 멀티 에이전트 시스템의 기초를 완성했습니다.

이제 다음 노트북에서 본격적인 에이전트 구현으로 넘어갑니다.

---

---

## 🎓 Part 2 & 3 학습 정리

### ✅ 지금까지 배운 핵심 내용

#### 1. ReACT 패턴 (Reasoning + Acting)
- **Thought**: AI가 다음 행동을 계획
- **Action**: 도구를 사용하여 행동
- **Observation**: 결과를 확인하고 다음 단계 결정
- 반복적으로 사고하고 행동하여 문제 해결

#### 2. BaseAgent 클래스
- 모든 에이전트의 기반이 되는 추상 클래스
- ReACT 패턴을 구현한 `run()` 메서드 제공
- LLM과 도구를 연결하는 핵심 로직

#### 3. 전문 에이전트들
- **PlannerAgent**: 학습 계획 수립 전문
- **NoteWriterAgent**: 학습 자료 작성 전문
- **AdvisorAgent**: 학습 조언 제공 전문

각 에이전트는 자신만의 system_prompt를 가지며,  
동일한 BaseAgent 로직을 재사용합니다.

---

### 🚀 다음 단계: 어디서 계속할까?

이 노트북에서는 **기초와 개별 에이전트**까지 구현했습니다.

완전한 멀티 에이전트 시스템을 구축하려면:

1. **Coordinator Agent** 구현
   - 여러 에이전트를 조율하는 마스터 에이전트
   - 사용자 요청을 분석하고 적절한 에이전트에 위임

2. **LangGraph로 워크플로우 구성**
   - 에이전트 간 데이터 흐름 정의
   - 상태 관리 및 전환 로직
   - 그래프 기반 실행 흐름

3. **전체 시스템 통합**
   - 모든 에이전트를 하나의 시스템으로 연결
   - End-to-end 테스트

4. **실제 API 통합**
   - Google Calendar API
   - Notion API
   - 실시간 데이터 활용

---

### 💡 핵심 개념 복습

```python
# ReACT 패턴 예시
while not done:
    # 1. Thought: 생각
    thought = "캘린더를 확인해야겠다"
    
    # 2. Action: 행동
    action = search_calendar(query="시험")
    
    # 3. Observation: 관찰
    observation = "알고리즘 시험이 3일 후입니다"
    
    # 4. 다음 Thought로 이어짐
    thought = "시험 준비 계획을 세워야겠다"
```

이것이 ReACT 패턴의 핵심입니다!

---

### 📚 참고: 기존 완전 구현본

더 완전한 구현을 보려면 `Academic_Task_Learning_Agent_LangGraph.ipynb` 파일을 참고하세요.

해당 파일에는 다음 내용이 포함되어 있습니다:
- Coordinator Agent 구현
- LangGraph 워크플로우
- 전체 시스템 통합
- 실제 API 통합 예시

---

### 🎉 축하합니다!

ReACT 패턴과 멀티 에이전트 시스템의 기초를 완성했습니다!

이제 다음 단계로 넘어가거나, 직접 에이전트를 확장해보세요.

---

In [ ]:
# ========================================
# 전문 에이전트 인스턴스 생성
# ========================================

# Planner 에이전트
planner = PlannerAgent(llm=llm, tools=tools)

# NoteWriter 에이전트
note_writer = NoteWriterAgent(llm=llm, tools=tools)

# Advisor 에이전트
advisor = AdvisorAgent(llm=llm, tools=tools)

print("✅ 모든 전문 에이전트 인스턴스 생성 완료\n")
print(f"📅 {planner.name}: 준비 완료")
print(f"📝 {note_writer.name}: 준비 완료")
print(f"💡 {advisor.name}: 준비 완료")

In [ ]:
# ========================================
# 1. PlannerAgent: 학습 계획 전문가
# ========================================

class PlannerAgent(BaseAgent):
    """
    학습 계획 및 일정 관리 전문 에이전트
    
    주요 기능:
    - 캘린더 이벤트 분석
    - 할 일 목록 우선순위 지정
    - 학습 계획 수립
    - 시간 관리 조언
    """
    
    def __init__(self, llm: NeMoLLaMa, tools: Tools):
        system_prompt = """당신은 학습 계획 및 시간 관리 전문가입니다.

역할:
1. 학생의 일정을 분석하여 중요한 이벤트를 파악합니다
2. 할 일 목록의 우선순위를 지정합니다
3. 효과적인 학습 계획을 수립합니다
4. 시험, 과제 마감일 등을 고려한 일정을 제안합니다

원칙:
- 항상 도구를 사용하여 최신 정보를 확인하세요
- 우선순위는 마감일과 중요도를 함께 고려하세요
- 학생의 학습 스타일을 반영하세요
- 구체적이고 실행 가능한 계획을 제시하세요"""

        super().__init__(
            name="PlannerAgent",
            llm=llm,
            tools=tools,
            system_prompt=system_prompt
        )


# ========================================
# 2. NoteWriterAgent: 학습 자료 작성 전문가
# ========================================

class NoteWriterAgent(BaseAgent):
    """
    학습 자료 생성 및 요약 전문 에이전트
    
    주요 기능:
    - 개념 설명 및 요약
    - 예제 제공
    - 학습 노트 작성
    - 복습 자료 생성
    """
    
    def __init__(self, llm: NeMoLLaMa, tools: Tools):
        system_prompt = """당신은 학습 자료 작성 전문가입니다.

역할:
1. 복잡한 개념을 쉽게 설명합니다
2. 구체적인 예제를 제공합니다
3. 체계적인 학습 노트를 작성합니다
4. 효과적인 복습 자료를 만듭니다

원칙:
- 학생의 수준에 맞게 설명하세요
- 시각적 학습자를 위한 다이어그램이나 표를 제안하세요
- 핵심 개념을 강조하세요
- 실생활 예제를 포함하세요"""

        super().__init__(
            name="NoteWriterAgent",
            llm=llm,
            tools=tools,
            system_prompt=system_prompt
        )


# ========================================
# 3. AdvisorAgent: 학습 조언 전문가
# ========================================

class AdvisorAgent(BaseAgent):
    """
    개인화된 학습 조언 전문 에이전트
    
    주요 기능:
    - 학습 방법 개선 제안
    - 동기부여 및 격려
    - 학습 전략 조언
    - 피드백 제공
    """
    
    def __init__(self, llm: NeMoLLaMa, tools: Tools):
        system_prompt = """당신은 학습 조언 및 멘토링 전문가입니다.

역할:
1. 학생의 학습 스타일을 분석합니다
2. 개인화된 학습 방법을 제안합니다
3. 동기부여와 격려를 제공합니다
4. 학습 전략과 습관 개선을 조언합니다

원칙:
- 학생 프로필을 참고하여 개인화된 조언을 하세요
- 긍정적이고 격려하는 태도를 유지하세요
- 과학적 근거가 있는 학습 방법을 추천하세요
- 실천 가능한 구체적인 조언을 제공하세요"""

        super().__init__(
            name="AdvisorAgent",
            llm=llm,
            tools=tools,
            system_prompt=system_prompt
        )


print("✅ 전문 에이전트 클래스 정의 완료\n")
print("📅 PlannerAgent: 학습 계획 수립")
print("📝 NoteWriterAgent: 학습 자료 작성")
print("💡 AdvisorAgent: 학습 조언 제공")

---

# 🎯 Part 3: 전문 에이전트 구현

## 1️⃣4️⃣ 전문 에이전트들

### 각 에이전트의 역할

ATLAS 시스템은 3개의 전문 에이전트로 구성됩니다:

1. **PlannerAgent** 📅
   - 일정 관리 및 학습 계획 수립
   - 캘린더와 할 일 목록 분석
   - 우선순위 기반 계획 제안

2. **NoteWriterAgent** 📝
   - 학습 자료 생성 및 요약
   - 개념 설명 및 예제 제공
   - 학습 노트 작성

3. **AdvisorAgent** 💡
   - 개인화된 학습 조언
   - 학습 방법 개선 제안
   - 동기부여 및 피드백

---

In [ ]:
# ========================================
# BaseAgent 실행 테스트 (선택사항)
# ========================================

# API 키가 설정되어 있다면 주석을 해제하고 실행하세요
# result = await test_agent.run(
#     state=initial_state,
#     user_input="다음 주에 예정된 중요한 일정을 알려주세요."
# )
# 
# print("\n" + "="*60)
# print("[Final Answer]")
# print("="*60)
# print(result)

print("⚠️  API 키를 설정한 후 위 주석을 해제하여 테스트를 실행하세요.")

In [ ]:
# ========================================
# BaseAgent 간단 테스트
# ========================================

# 테스트 에이전트 생성
test_agent = BaseAgent(
    name="TestAgent",
    llm=llm,
    tools=tools,
    system_prompt="당신은 학업 관리를 돕는 AI 비서입니다. 학생의 일정과 과제를 관리합니다."
)

# 초기 상태 생성
initial_state: AcademicState = {
    "messages": [],
    "profile": sample_profile,
    "calendar": sample_calendar,
    "tasks": sample_tasks,
    "results": {}
}

# NOTE: 이 셀을 실행하려면 위에서 API 키를 올바르게 설정해야 합니다!
# 만약 API 키가 설정되지 않았다면 이 셀은 건너뛰세요.

print("✅ 테스트 에이전트 생성 완료")
print(f"에이전트 이름: {test_agent.name}")
print("\n⚠️  실제 테스트를 실행하려면 아래 셀의 주석을 해제하세요.")

## 1️⃣3️⃣ BaseAgent 테스트

### 🧪 간단한 테스트로 ReACT 패턴 확인

BaseAgent가 제대로 작동하는지 간단한 테스트 에이전트를 만들어 확인합니다.

---

In [ ]:
# ========================================
# BaseAgent: 모든 에이전트의 기본 클래스
# ========================================

class BaseAgent:
    """
    모든 에이전트의 기본이 되는 추상 클래스
    
    ReACT 패턴을 구현하며, 각 전문 에이전트는 이 클래스를 상속합니다.
    
    핵심 개념:
    - Thought (사고): 무엇을 해야 할지 생각
    - Action (행동): 도구를 사용하여 행동
    - Observation (관찰): 행동의 결과 확인
    
    사용 예시:
        class PlannerAgent(BaseAgent):
            def __init__(self, llm, tools):
                super().__init__(
                    name="Planner",
                    llm=llm,
                    tools=tools,
                    system_prompt="당신은 학습 계획 전문가입니다..."
                )
    """
    
    def __init__(
        self,
        name: str,
        llm: NeMoLLaMa,
        tools: Tools,
        system_prompt: str
    ):
        """
        에이전트 초기화
        
        Args:
            name: 에이전트 이름
            llm: LLM 인스턴스
            tools: 사용 가능한 도구들
            system_prompt: 에이전트의 역할을 정의하는 프롬프트
        """
        self.name = name
        self.llm = llm
        self.tools = tools
        self.system_prompt = system_prompt
        self.max_iterations = 5  # 무한 루프 방지
    
    def _build_prompt(self, state: AcademicState, user_input: str) -> List[Dict]:
        """
        프롬프트 구성
        
        ReACT 패턴을 위한 프롬프트를 만듭니다.
        에이전트가 사고하고 행동할 수 있도록 안내합니다.
        """
        # 컨텍스트 정보 구성
        context = self._build_context(state)
        
        # 도구 목록
        available_tools = """
사용 가능한 도구:
1. search_calendar(query=""): 캘린더에서 이벤트 검색
2. search_tasks(status="needsAction"): 할 일 목록 검색
3. get_profile(field=""): 학생 프로필 조회
4. calculate_time_until(event_time): 이벤트까지 남은 시간 계산
"""
        
        # ReACT 형식 지침
        react_format = """
다음 형식으로 응답하세요:

Thought: [무엇을 해야 할지 생각]
Action: [사용할 도구 이름]
Action Input: [도구에 전달할 파라미터 (JSON 형식)]

또는 작업이 완료되면:

Final Answer: [최종 답변]
"""
        
        messages = [
            {
                "role": "system",
                "content": f"{self.system_prompt}\n\n{available_tools}\n\n{react_format}"
            },
            {
                "role": "user",
                "content": f"컨텍스트:\n{context}\n\n요청: {user_input}"
            }
        ]
        
        return messages
    
    def _build_context(self, state: AcademicState) -> str:
        """
        상태로부터 컨텍스트 정보 추출
        """
        profile = state.get("profile", {})
        student_name = profile.get("personal_info", {}).get("name", "학생")
        
        context = f"학생 이름: {student_name}\n"
        
        # 최근 대화 히스토리 추가 (마지막 3개)
        messages = state.get("messages", [])
        if messages:
            context += "\n최근 대화:\n"
            for msg in messages[-3:]:
                if isinstance(msg, HumanMessage):
                    context += f"사용자: {msg.content}\n"
                elif isinstance(msg, AIMessage):
                    context += f"AI: {msg.content}\n"
        
        return context
    
    def _parse_action(self, response: str) -> Optional[Dict]:
        """
        LLM 응답에서 Action 파싱
        
        Returns:
            {"action": "도구이름", "input": {...}} 또는 None
        """
        lines = response.strip().split("\n")
        
        action_name = None
        action_input = {}
        
        for line in lines:
            line = line.strip()
            
            if line.startswith("Action:"):
                action_name = line.replace("Action:", "").strip()
            
            elif line.startswith("Action Input:"):
                input_str = line.replace("Action Input:", "").strip()
                try:
                    action_input = json.loads(input_str)
                except:
                    action_input = {"query": input_str}
        
        if action_name:
            return {"action": action_name, "input": action_input}
        
        return None
    
    def _execute_action(self, action_dict: Dict) -> str:
        """
        도구 실행
        
        Args:
            action_dict: {"action": "도구이름", "input": {...}}
        
        Returns:
            도구 실행 결과 (문자열)
        """
        action_name = action_dict.get("action", "")
        action_input = action_dict.get("input", {})
        
        # 도구 이름을 메서드 이름으로 변환
        if hasattr(self.tools, action_name):
            tool_method = getattr(self.tools, action_name)
            
            try:
                result = tool_method(**action_input)
                return json.dumps(result, ensure_ascii=False, indent=2)
            except Exception as e:
                return f"도구 실행 오류: {str(e)}"
        
        return f"알 수 없는 도구: {action_name}"
    
    async def run(self, state: AcademicState, user_input: str) -> str:
        """
        에이전트 실행 - ReACT 패턴의 핵심
        
        Args:
            state: 현재 상태
            user_input: 사용자 입력
        
        Returns:
            최종 답변
        """
        messages = self._build_prompt(state, user_input)
        iteration = 0
        
        while iteration < self.max_iterations:
            iteration += 1
            
            # LLM 호출
            response = await self.llm.agenerate(messages, temperature=0.6)
            
            print(f"\n{'='*60}")
            print(f"[{self.name}] Iteration {iteration}")
            print(f"{'='*60}")
            print(response)
            
            # Final Answer가 있으면 종료
            if "Final Answer:" in response:
                final_answer = response.split("Final Answer:")[-1].strip()
                return final_answer
            
            # Action 파싱
            action_dict = self._parse_action(response)
            
            if action_dict:
                # 도구 실행
                observation = self._execute_action(action_dict)
                
                print(f"\n[Observation]")
                print(observation)
                
                # 대화에 추가
                messages.append({
                    "role": "assistant",
                    "content": response
                })
                messages.append({
                    "role": "user",
                    "content": f"Observation: {observation}\n\n계속 진행하세요."
                })
            else:
                # Action을 파싱하지 못했으면 그냥 응답 반환
                return response
        
        return "최대 반복 횟수에 도달했습니다. 작업을 완료하지 못했습니다."

print("✅ BaseAgent 클래스 정의 완료")
print("\nBaseAgent는 모든 전문 에이전트의 기반입니다.")
print("ReACT 패턴 (Thought → Action → Observation)을 구현합니다.")

## 1️⃣2️⃣ BaseAgent 클래스 구현

### 🏗️ 모든 에이전트의 기반

BaseAgent는 모든 전문 에이전트(Planner, NoteWriter, Advisor)가 공통으로 사용할 기능을 제공합니다.

#### 핵심 기능
1. **System Prompt**: 에이전트의 역할 정의
2. **Tools**: 사용 가능한 도구 목록
3. **run()**: ReACT 패턴 실행 메서드

---

In [ ]:
# ========================================
# 도구(Tools) 정의
# ========================================

class Tools:
    """
    에이전트가 사용할 수 있는 도구 모음
    
    각 도구는 특정 작업을 수행하는 메서드입니다.
    실제 환경에서는 외부 API를 호출하겠지만,
    학습을 위해 샘플 데이터를 반환합니다.
    """
    
    def __init__(self, calendar_data: Dict, tasks_data: Dict, profile_data: Dict):
        """
        도구 초기화
        
        Args:
            calendar_data: 캘린더 이벤트 데이터
            tasks_data: 할 일 목록 데이터
            profile_data: 학생 프로필 데이터
        """
        self.calendar = calendar_data
        self.tasks = tasks_data
        self.profile = profile_data
    
    def search_calendar(self, query: str = "") -> Dict:
        """
        캘린더에서 이벤트 검색
        
        실제로는 Google Calendar API를 호출
        """
        events = self.calendar.get("events", [])
        
        # 검색어가 있으면 필터링
        if query:
            events = [e for e in events if query.lower() in e["summary"].lower()]
        
        return {
            "success": True,
            "events": events,
            "count": len(events)
        }
    
    def search_tasks(self, status: str = "needsAction") -> Dict:
        """
        할 일 목록 검색
        
        실제로는 Google Tasks 또는 Notion API를 호출
        """
        tasks = self.tasks.get("tasks", [])
        
        # 상태로 필터링
        if status:
            tasks = [t for t in tasks if t.get("status") == status]
        
        # 우선순위로 정렬
        priority_order = {"high": 0, "medium": 1, "low": 2}
        tasks_sorted = sorted(tasks, key=lambda x: priority_order.get(x.get("priority", "low"), 3))
        
        return {
            "success": True,
            "tasks": tasks_sorted,
            "count": len(tasks_sorted)
        }
    
    def get_profile(self, field: str = "") -> Dict:
        """
        학생 프로필 정보 조회
        
        Args:
            field: 특정 필드만 가져오기 (예: "learning_preferences")
        """
        if field and field in self.profile:
            return {
                "success": True,
                "data": {field: self.profile[field]}
            }
        
        return {
            "success": True,
            "data": self.profile
        }
    
    def calculate_time_until(self, event_time: str) -> Dict:
        """
        특정 이벤트까지 남은 시간 계산
        
        Args:
            event_time: ISO 형식의 시간 문자열
        """
        try:
            event_dt = datetime.fromisoformat(event_time.replace("Z", "+00:00"))
            now = datetime.now(timezone.utc)
            delta = event_dt - now
            
            days = delta.days
            hours = delta.seconds // 3600
            
            return {
                "success": True,
                "days": days,
                "hours": hours,
                "total_hours": days * 24 + hours
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }

# 도구 인스턴스 생성
tools = Tools(sample_calendar, sample_tasks, sample_profile)

print("✅ 도구(Tools) 클래스 정의 완료")
print("\n사용 가능한 도구:")
print("  - search_calendar(): 캘린더 이벤트 검색")
print("  - search_tasks(): 할 일 목록 검색")
print("  - get_profile(): 학생 프로필 조회")
print("  - calculate_time_until(): 이벤트까지 남은 시간 계산")

---

# 🚀 Part 2: ReACT Agent 패턴 구현

## 1️⃣1️⃣ ReACT 패턴이란?

### 🧠 ReACT = Reasoning + Acting

ReACT는 AI 에이전트가 **사고(Reasoning)**와 **행동(Acting)**을 반복하며 문제를 해결하는 패턴입니다.

#### 전통적인 방식의 문제점
```python
# 한 번에 모든 것을 처리 시도
response = llm("학습 계획을 세워주세요")
# 문제: 현재 일정을 모르고, 과제 마감일도 모르는 상태에서 답변
```

#### ReACT 패턴의 해결책
```
1. Thought: "먼저 학생의 일정을 확인해야겠다"
2. Action: search_calendar()
3. Observation: "알고리즘 시험이 3일 후, 과제는 내일 마감"
4. Thought: "긴급한 과제부터 처리하도록 계획해야겠다"
5. Action: create_plan()
6. Final Answer: "내일까지 과제 완료 후, 시험 준비에 집중하세요"
```

### 📊 ReACT 순환 구조

```
┌─────────────┐
│   Thought   │ ← "무엇을 해야 할까?"
│  (사고)     │
└──────┬──────┘
       │
       ▼
┌─────────────┐
│   Action    │ ← "도구를 사용하자"
│  (행동)     │
└──────┬──────┘
       │
       ▼
┌─────────────┐
│ Observation │ ← "결과를 확인했다"
│  (관찰)     │
└──────┬──────┘
       │
       └──────► (다음 Thought로 돌아감)
```

---